In [1]:
import pycosat

In [25]:
def gen_all_clauses(k, n):
    if k == 0:
        yield tuple()
    else:
        for clause in gen_all_clauses(k-1, n):
            for var in range(1, n+1):
                for svar in [var, -var]:
                    yield clause + (svar,)

def gen_all_kcnf(k, n, m):
    if m == 0:
        yield tuple()
    else:
        for cnf in gen_all_kcnf(k, n, m-1):
            for clause in gen_all_clauses(k, n):
                yield cnf + (clause,)

In [28]:
def filter_satable(cnf_gen):
    for cnf in cnf_gen:
        if pycosat.solve(cnf) != 'UNSAT':
            yield cnf

def filter_satable_by(cnf_gen, solution):
    # WARNING: it assumes solution is full
    solution_clauses = tuple([svar] for svar in solution)
    for cnf in cnf_gen:
        if pycosat.solve(cnf + solution_clauses) != 'UNSAT':
            yield cnf

In [135]:
def gen_satable_by_clause(k, n, solution):
    if k == 0:
        return
    else:
        for svar in solution:
            for clause in gen_all_clauses(k-1, n):
                yield clause + (svar,)
        for clause in gen_satable_by_clause(k-1, n, solution):
            for svar in solution:
                yield clause + (-svar,)

def gen_satable_by_kcnf(k, n, m, solution):
    if m == 0:
        yield tuple()
    else:
        for cnf in gen_satable_by_kcnf(k, n, m-1, solution):
            for clause in gen_satable_by_clause(k, n, solution):
                yield cnf + (clause,)

In [150]:
K = 2
VARS = 1
CLAUSES = 2
SOLUTION = [x for x in range(1, VARS+1)]

all_possible = tuple(sorted(list(gen_all_kcnf(K, VARS, CLAUSES))))
all_satable = tuple(filter_satable(all_possible))
all_satable_by = tuple(filter_satable_by(all_possible, SOLUTION))
print("Possible:", len(all_possible))
print("Satable:", len(all_satable))
print("Satable by:", len(all_satable_by))
print((2**K - 1) * VARS**K)
print((2**K) * VARS**K)

Possible: 16
Satable: 14
Satable by: 9
3
4


In [138]:
all_satable_by = tuple(sorted(tuple(gen_satable_by_kcnf(K, VARS, CLAUSES, SOLUTION))))
print("Satable by:", len(all_satable_by))

Satable by: 448


# Spam

In [132]:
K = 3
VARS = 1
CLAUSES = 1
SOLUTION = [x for x in range(1, VARS+1)]

all_possible = tuple(sorted(list(gen_all_kcnf(K, VARS, CLAUSES))))
all_satable = tuple(filter_satable(all_possible))
print("Possible:", len(all_possible))
print("Satable:", len(all_satable))
print()
print(((2*VARS)**K)**CLAUSES)
print(2*VARS)

Possible: 8
Satable: 8

8
2
